In [1]:
"""In this example, we use the public comma car segments database to check if vin fingerprinting is feasible for ford."""

from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database
from openpilot.selfdrive.car.ford.values import CAR

database = get_comma_car_segments_database()

platforms = [c.value for c in CAR]

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/jnewb/AppData/Local/Programs/Microsoft VS Code


In [2]:
# Adapted from https://github.com/commaai/openpilot/issues/31052#issuecomment-1902690083

MODEL_YEAR_CODES = {'M': 2021, 'N': 2022, 'P': 2023, 'R': 2024, 'S': 2025}


F150_CODES = ['F1C', 'F1E', 'W1C', 'W1E', 'X1C', 'X1E', 'W1R', 'W1P', 'W1S', 'W1T']
LIGHTNING_CODES = ['L', 'V']
MACHE_CODES = ['K1R', 'K1S', 'K2S', 'K3R', 'K3S', 'K4S']

FORD_VIN_START = ['1FT', '3FM', '5LM']

def ford_vin_fingerprint(vin): # Check if it's a Ford vehicle and determine the model
  vin_positions_567 = vin[4:7]

  if vin.startswith('1FT'):
    if vin_positions_567 in F150_CODES:
      if vin[7] in LIGHTNING_CODES:
        return f"FORD F-150 LIGHTNING 1ST GEN"
      else:
        return f"FORD F-150 14TH GEN"
  elif vin.startswith('3FM'):
    if vin_positions_567 in MACHE_CODES:
      return f"FORD MUSTANG MACH-E 1ST GEN"
  elif vin.startswith('5LM'):
    pass

  return "mock"

In [3]:
from openpilot.tools.lib.logreader import get_first_message


VINS_TO_CHECK = set()

for platform in platforms:
  if platform not in database:
    print(f"Skipping platform: {platform}, no data available")
    continue

  for segment in database[platform]:
    lr = LogReader(segment)
    CP = get_first_message(lr, "carParams").carParams
    VINS_TO_CHECK.add((CP.carVin, CP.carFingerprint))

Skipping platform: FORD F-150 14TH GEN, no data available


In [4]:
for vin, real_fingerprint in VINS_TO_CHECK:
  determined_fingerprint = ford_vin_fingerprint(vin)
  print(f"vin: {vin} real platform: {real_fingerprint: <30}     determined platform: {determined_fingerprint: <30}    correct: {real_fingerprint == determined_fingerprint}")

vin: 3FTTW8E31PRXXXXXX real platform: FORD MAVERICK 1ST GEN              determined platform: mock                              correct: False
vin: 1FM5K8GC7LGXXXXXX real platform: FORD EXPLORER 6TH GEN              determined platform: mock                              correct: False
vin: 3FTTW8E99NRXXXXXX real platform: FORD MAVERICK 1ST GEN              determined platform: mock                              correct: False
vin: 1FM5K8GC7NGXXXXXX real platform: FORD EXPLORER 6TH GEN              determined platform: mock                              correct: False
vin: 1FM5K7LC0MGXXXXXX real platform: FORD EXPLORER 6TH GEN              determined platform: mock                              correct: False
vin: 1FM5K8HC7MGXXXXXX real platform: FORD EXPLORER 6TH GEN              determined platform: mock                              correct: False
vin: 3FMTK3SU0MMXXXXXX real platform: FORD MUSTANG MACH-E 1ST GEN        determined platform: FORD MUSTANG MACH-E 1ST GEN       correct: True
